In [16]:
import cv2
import os
import numpy as np
from tqdm import tqdm

TEST_DIR  ='your testing directory'
IMG_SIZE= 50

LR= 1e-3
MODEL_NAME= 'Automatic_Checker_System-{}-{}.model'.format(LR,'4_conv-basic')

In [17]:
## Specifying the already defined model on which the training was done

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()


convnet= input_data(shape=[None,IMG_SIZE,IMG_SIZE,1], name= 'input')

convnet= conv_2d(convnet,32,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,64,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,96,2, activation='relu')
convnet= max_pool_2d(convnet,2)

convnet= conv_2d(convnet,128,2, activation='relu')
convnet= max_pool_2d(convnet,2)


convnet= fully_connected(convnet, 200 , activation='softmax')
convnet= regression(convnet, optimizer='adam', learning_rate=LR,loss='categorical_crossentropy', name='targets')

model =tflearn.DNN(convnet, tensorboard_dir='log')

In [18]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

INFO:tensorflow:Restoring parameters from C:\Users\Dev\MachineLearningProjects\Automatic_Checker_System-0.001-4_conv-basic.model
model loaded!


In [19]:
## capturing the video feed

webcam = cv2.VideoCapture('output_.mp4') 

## Haar cascades for detecting the frontal faces in the video
classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

while webcam.isOpened():
    (rval, im) = webcam.read()
    if rval== False:
        break
    faces= classifier.detectMultiScale(im)
    for (x,y,w,h) in faces:
        sub_face= im[y:y+h, x:x+w]
        filename= str(y)+".jpg"
        cv2.imwrite(filename, sub_face)

    if cv2.waitKey(1) & 0xff== ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

In [20]:
## copying the images to the test directory

import glob
import shutil


src_dir = "C:\\Users\\Dev\\MachineLearningProjects"
dst_dir = "C:\\Users\\Dev\\MachineLearningProjects\\Test_images"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)


In [21]:
def process_test_data():
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        path= os.path.join(TEST_DIR,img)
        img_num= img.split('.')[0]
        img= cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE),interpolation=cv2.INTER_CUBIC)
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy',testing_data)
    return testing_data

In [22]:
## Collecting the testing data in a list


test_data= process_test_data()



for num,data in enumerate(test_data[:11]):
    
    im_num=data[1]
    im_data= np.array(data[0])

    data= im_data.reshape(IMG_SIZE,IMG_SIZE,1)
    
    model_out= model.predict([data])
    print(np.argmax(model_out[0])+1,im_num)

    
    

100%|█████████████████████████████████████████| 30/30 [00:00<00:00, 468.87it/s]


1 100
1 101
126 102
9 103
9 104
13 105
13 106
13 107
3 108
14 115
14 116


In [ ]:
## The model is predicting on test data with an accuracy of 85%